In [ ]:
# Copyright 2022 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table align="left">
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/community/prediction/custom_prediction_routines/SDK_Custom_Preprocess.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
    <a href="https://console.cloud.google.com/vertex-ai/notebooks/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/community/prediction/custom_prediction_routines/SDK_Custom_Preprocess.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>
</table>

## 概述

本教程演示了如何使用Vertex AI SDK构建一个自定义容器，该容器使用Custom Prediction Routine模型服务器来为Vertex AI Predictions提供scikit-learn模型的服务。



### 数据集

本教程使用了R.A. Fisher的鸢尾花数据集，这是一个小型数据集，非常适合尝试机器学习技术。每个实例都有四个数值特征，这些特征是对一朵花的不同测量值，还有一个目标标签，表示它是三种类型中的一种：鸢尾花(Iris setosa)、变色鸢尾花(Iris versicolour)或维吉尼亚鸢尾花(Iris virginica)。

本教程使用了[scikit-learn库中包含的鸢尾花数据集的副本](https://scikit-learn.org/stable/auto_examples/datasets/plot_iris_dataset.html)。

### 目标

目标是：
- 训练一个模型，使用花朵的测量值作为输入来预测它是哪种类型的鸢尾花。
- 保存模型及其序列化的预处理器
- 使用Vertex AI SDK中的自定义预测例程特性构建一个自定义sklearn服务容器，进行自定义预处理
- 在本地测试构建的容器
- 上传并部署自定义容器到Vertex Prediction

本教程更注重于将此模型部署到Vertex AI上，而不是模型本身的设计。

### 成本

本教程使用Google Cloud的可计费组件：

* Vertex AI

了解[Vertex AI的定价](https://cloud.google.com/vertex-ai/pricing)，并使用[Pricing Calculator](https://cloud.google.com/products/calculator/)基于您的预计使用情况生成成本估算。

### 设置您的本地开发环境

**如果您正在使用 Vertex AI Workbench 笔记本**，您的环境已经符合运行此笔记本的所有要求。您可以跳过此步骤。

否则，请确保您的环境符合本笔记本的要求。
您需要以下内容：

* Docker
* Git
* Google Cloud SDK（gcloud）
* Python 3
* virtualenv
* 在带有Python 3的虚拟环境中运行的Jupyter笔记本

Google Cloud指南[设置Python开发环境](https://cloud.google.com/python/setup)和[Jupyter安装指南](https://jupyter.org/install)提供了满足这些要求的详细说明。以下步骤提供了简明的一组指令：

1. [安装和初始化Cloud SDK。](https://cloud.google.com/sdk/docs/)

1. [安装Python 3。](https://cloud.google.com/python/setup#installing_python)

1. [安装virtualenv](https://cloud.google.com/python/setup#installing_and_using_virtualenv)并创建一个使用Python 3的虚拟环境。激活虚拟环境。

1. 要安装Jupyter，请在终端窗口中运行`pip install jupyter` 命令。

1. 要启动Jupyter，请在终端窗口中运行`jupyter notebook` 命令。

1. 在Jupyter Notebook仪表板中打开这个笔记本。

安装额外的包

安装额外的包依赖项，这些包在您的笔记本环境中尚未安装，例如NumPy，Scikit-learn，FastAPI，Uvicorn和joblib。

In [ ]:
%%writefile requirements.txt
fastapi
uvicorn==0.17.6
joblib~=1.0
numpy~=1.20
scikit-learn~=0.24
google-cloud-storage>=1.26.0,<2.0.0dev
google-cloud-aiplatform[prediction]>=1.16.0

你部署的模型将预先安装有不同的依赖项集，而笔记本环境中没有。你不应该假设因为在笔记本中可以运行，所以在模型中也会运行。相反，你应该通过在requirements.txt中列出依赖项的方式对模型的依赖性进行非常明确的说明，然后使用`pip install`在笔记本中安装完全相同的依赖项。请注意，当然也有可能在requirements.txt中遗漏了笔记本中已经存在的依赖项。如果是这种情况，那么事情将会在笔记本中运行，但在模型中不会。为了防范这种情况，你将在部署到云端之前在本地测试模型。

In [ ]:
# Install the same dependencies used in the serving container in the notebook
# environment.
%pip install -U --user -r requirements.txt

### 重新启动内核

在安装了额外的包之后，您需要重新启动笔记本内核，以便它可以找到这些包。

In [ ]:
# Automatically restart kernel after installs
import os

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

在开始之前

在这个示例中设置日志记录

In [ ]:
import logging

logging.basicConfig(level=logging.INFO)

### 设置您的谷歌云项目

**无论您使用哪种笔记本环境，都需要执行以下步骤。**

1. [选择或创建谷歌云项目](https://console.cloud.google.com/cloud-resource-manager)。当您首次创建帐户时，您将获得$300的免费信用，可用于支付计算/存储成本。

1. [确保您的项目已启用计费](https://cloud.google.com/billing/docs/how-to/modify-project)。

1. [启用 Vertex AI API 和 Compute Engine API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com,compute_component)。

1. 如果您在本地运行这个笔记本，您需要安装[Cloud SDK](https://cloud.google.com/sdk)。

1. 在下面的单元格中输入您的项目ID。然后运行该单元格，以确保Cloud SDK在这个笔记本中的所有命令中使用正确的项目。。

**注意**：Jupyter会将以`!`或`%`为前缀的行视为shell命令，并将Python变量插入这些命令中使用`$`或`{}`。

将您的项目ID设置为

**如果您不知道您的项目ID**，您可以使用 `gcloud` 获取您的项目ID。

In [ ]:
import os

PROJECT_ID = ""

# Get your Google Cloud project ID from gcloud
if not os.getenv("IS_TESTING"):
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID: ", PROJECT_ID)

否则，在这里设置您的项目ID。

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None:
    PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
! gcloud config set project $PROJECT_ID

#### 地区

您还可以更改“REGION”变量，该变量用于笔记本其余部分的操作。以下是Vertex AI支持的地区。我们建议您选择距离您最近的地区。

- 美洲：`us-central1`
- 欧洲：`europe-west4`
- 亚太地区：`asia-east1`

您不能在Vertex AI上使用多地区存储桶进行训练。并非所有地区都支持所有Vertex AI服务。

了解更多关于[Vertex AI地区](https://cloud.google.com/vertex-ai/docs/general/locations)。

In [ ]:
REGION = "[your-region]"  # @param {type: "string"}

if REGION == "[your-region]":
    REGION = "us-central1"

### 配置项目和资源名称

如果你在一个实时教程会话中，你可能正在使用一个共享的测试账号或项目。为了避免用户之间在创建的资源上发生名称冲突，你可以为每个实例会话创建一个时间戳，并将其附加到你在本教程中创建的资源的名称上。

In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

配置GCP资源名称。

In [ ]:
MODEL_ARTIFACT_DIR = "sklearn-cpr-preprocess-model"  # @param {type:"string"}
REPOSITORY = "custom-preprocess-container-prediction"  # @param {type:"string"}
IMAGE = "sklearn-cpr-preprocess-server"  # @param {type:"string"}
MODEL_DISPLAY_NAME = "sklearn-cpr-preprocess-model"  # @param {type:"string"}

`MODEL_ARTIFACT_DIR` - 模型工件所在的文件夹目录路径，位于 Cloud 存储桶内，例如："my-models/fraud-detection/trial-4"

`REPOSITORY` - 要创建或使用的工件存储库的名称。

`IMAGE` - 要推送的容器映像的名称。

`MODEL_DISPLAY_NAME` - Vertex AI 模型资源的显示名称。

### 创建一个云存储桶

**无论您使用的是哪种笔记本环境，都需要执行以下步骤。**

如果要更新模型制品，而不需要重新构建容器，您必须将模型制品和任何自定义代码上传到云存储中。

在下面设置您的云存储桶的名称。它必须在所有云存储桶中是唯一的。

In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}
BUCKET_URI = f"gs://{BUCKET_NAME}"

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "-aip-" + TIMESTAMP
    BUCKET_URI = f"gs://{BUCKET_NAME}"

只有在您的存储桶不存在时才运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

最后，通过检查您的云存储存储桶的内容来验证访问权限。

In [ ]:
! gsutil ls -al $BUCKET_URI

编写您的预处理器

把目录设置到放置你所有自定义代码的地方。

In [ ]:
USER_SRC_DIR = "src_dir"  # @param {type:"string"}

决定放置训练模型的目录。

In [ ]:
LOCAL_MODEL_ARTIFACTS_DIR = "model_artifacts"  # @param {type:"string"}

将训练数据进行缩放，使得每个数值特征列的平均值为0，标准差为1，可以改善您的模型。

创建一个名为`preprocess.py`的文件，其中包含一个用于执行此缩放操作的类。

In [ ]:
%mkdir $USER_SRC_DIR
%mkdir $LOCAL_MODEL_ARTIFACTS_DIR

In [ ]:
%%writefile $USER_SRC_DIR/preprocess.py
import numpy as np

class MySimpleScaler(object):
    def __init__(self):
        self._means = None
        self._stds = None

    def preprocess(self, data):
        if self._means is None:  # during training only
            self._means = np.mean(data, axis=0)

        if self._stds is None:  # during training only
            self._stds = np.std(data, axis=0)
            if not self._stds.all():
                raise ValueError("At least one column has standard deviation of 0.")

        return (data - self._means) / self._stds

## 使用预处理器训练和存储模型
接下来，使用`preprocess.MySimpleScaler`对鸢尾花数据进行预处理，然后使用scikit-learn训练模型。

最后，将训练好的模型导出为joblib（`.joblib`）文件，并将`MySimpleScaler`实例导出为pickle（`.pkl`）文件：

In [ ]:
%cd $USER_SRC_DIR/

import pickle

import joblib
from preprocess import MySimpleScaler
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier

iris = load_iris()
scaler = MySimpleScaler()

X = scaler.preprocess(iris.data)
y = iris.target

model = RandomForestClassifier()
model.fit(X, y)

joblib.dump(model, f"../{LOCAL_MODEL_ARTIFACTS_DIR}/model.joblib")
with open(f"../{LOCAL_MODEL_ARTIFACTS_DIR}/preprocessor.pkl", "wb") as f:
    pickle.dump(scaler, f)

### 将模型工件和自定义代码上传到云存储

在部署模型进行服务之前，Vertex AI 需要访问以下文件在云存储中：

* `model.joblib`（模型工件）
* `preprocessor.pkl`（前处理器代码）

运行以下命令来上传您的文件：

In [ ]:
%cd ..
!gsutil cp {LOCAL_MODEL_ARTIFACTS_DIR}/* {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/
!gsutil ls {BUCKET_URI}/{MODEL_ARTIFACT_DIR}/

使用CPR模型服务器构建定制的服务容器

现在模型和处理器已经训练并保存好了，是时候构建自定义的服务容器了。通常构建一个服务容器需要编写模型服务器代码。然而，有了自定义预测例程功能，Vertex AI Prediction 提供了一个可以直接使用的模型服务器。

自定义的服务容器包含以下三个代码部分：
1. [模型服务器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/model_server.py)
    * 托管模型的HTTP服务器。
    * 负责设置路由/端口等。
1. [请求处理器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/handler.py)
    * 负责处理请求的网络服务器方面，例如对请求主体进行反序列化、序列化响应、设置响应头等。
    * 在本例中，您将使用SDK中提供的默认处理器`google.cloud.aiplatform.prediction.handler.PredictionHandler`。
1. [预测器](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/predictor.py)
    * 负责处理预测请求的机器学习逻辑。

这三个部分中的每一个都可以根据自定义容器的需求进行定制。在本例中，您只需实现`Predictor`。

您将使用预定义的[`SklearnPredictor`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/sklearn/predictor.py)作为`CprPredictor`的基类。您只需要实现`load`、`preprocess`和`postprocess`方法。

请注意，[`PredictionHandler`](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/prediction/handler.py)将用于处理预测请求，将执行以下操作：
```
self._predictor.postprocess(self._predictor.predict(self._predictor.preprocess(prediction_input)))
```

首先，实现一个自定义的`Predictor`，它加载预处理器。然后在`preprocess`时间使用处理器。

自定义预测例程支持一种方式，在本地运行容器来测试您的图像。在本地测试图像时，您可以传递一个GCS路径或本地路径。
- 如果传递了GCS路径，您需要设置凭证。
- 如果要通过传递本地路径测试，您需要在您的`Predictor`中支持远程和本地加载模型。

Vertex AI SDK提供了[`download_model_artifacts`函数](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py)来帮助您从GCS路径或本地路径下载模型工件。请参见下面`load`函数中的示例。`SklearnPredictor`中的`load`函数使用`download_model_artifacts`函数来准备模型工件。

In [ ]:
%%writefile $USER_SRC_DIR/predictor.py

import joblib
import numpy as np
import pickle

from google.cloud.aiplatform.prediction.sklearn.predictor import SklearnPredictor
from google.cloud.aiplatform.utils import prediction_utils

from sklearn.datasets import load_iris


class CprPredictor(SklearnPredictor):
    
    def __init__(self):
        return
    
    def load(self, artifacts_uri: str):
        """Loads the preprocessor artifacts."""
        super().load(artifacts_uri)

        with open("preprocessor.pkl", "rb") as f:
            preprocessor = pickle.load(f)

        self._class_names = load_iris().target_names
        self._preprocessor = preprocessor
    
    def preprocess(self, prediction_input):
        """Perform scaling preprocessing"""
        inputs = super().preprocess(prediction_input)
        return self._preprocessor.preprocess(inputs)
    
    def postprocess(self, prediction_results):
        """Convert class indices to class names."""
        return {"predictions": [self._class_names[class_num] for class_num in prediction_results]}

构建自定义容器时，您还需要编写启动模型服务器的映像入口点。但是，使用自定义预测例程功能，您不再需要编写入口点。Vertex AI SDK 将使用您提供的自定义预测器填充入口点。

写出源目录的依赖项，这些依赖项将安装到映像中。

In [ ]:
!cp requirements.txt $USER_SRC_DIR/requirements.txt

构建并推送容器到工件库

### 构建您的自定义容器

要构建自定义镜像，您需要一个 Dockerfile，在其中您需要实现镜像的外观。但是，使用自定义预测例程功能时，Vertex AI SDK 也会为您生成 Dockerfile 并构建镜像。

默认情况下，使用 `python:3.7` 作为基础镜像。

In [ ]:
import os

from google.cloud.aiplatform.prediction import LocalModel
from src_dir.predictor import \
    CprPredictor  # Update this path as the variable $USER_SRC_DIR to import the custom predictor.

local_model = LocalModel.build_cpr_model(
    USER_SRC_DIR,
    f"{REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}",
    predictor=CprPredictor,
    requirements_path=os.path.join(USER_SRC_DIR, "requirements.txt"),
)

您可以查看构建图像的服务容器规格。

In [ ]:
local_model.get_serving_container_spec()

### 存储测试实例

要了解如何在 JSON 中格式化输入实例，请阅读[文档](https://cloud.google.com/vertex-ai/docs/predictions/online-predictions-custom-models#request-body-details)。

In [ ]:
INPUT_FILE = "instances.json"

In [ ]:
%%writefile $INPUT_FILE
{
    "instances": [
        [6.7, 3.1, 4.7, 1.5],
        [4.6, 3.1, 1.5, 0.2]
    ]
}

### 本地运行和测试容器

自定义预测程序提供了两种下载模型文件的方式，用于在本地测试图像。
1. 本地路径。
2. GCS路径。

要使用本地路径，您的`Predictor`需要支持加载模型的两种方式，以便可以在本地进行测试并部署到Vertex AI预测服务。SDK提供了一个名为`download_model_artifacts`的方法，支持在[prediction_utils.py](https://github.com/googleapis/python-aiplatform/blob/main/google/cloud/aiplatform/utils/prediction_utils.py)中的这两种方式，您可以在`Predictor`的`load`函数中调用这个方法。

如果您想要使用GCS路径在本地测试图像，您需要遵循设置凭据的说明。

#### 设置凭证

设置凭证仅在本地运行带有GCS路径的自定义服务容器时才需要。设置凭证是为了执行`Predictor`的`load`函数，该函数从Google Cloud Storage下载模型文件。

要访问您项目中的Google Cloud Storage，您需要通过以下方式之一设置凭证：

1. 用户账户

2. 服务账户

您可以在[这里](https://cloud.google.com/docs/authentication#principals)了解更多关于上述每种凭证的信息。

选项1. 使用Google用户凭据

首先授权Google auth库以使用Google用户凭据访问Cloud平台。这一步涉及一个需要用户输入的交互提示。如果您正在使用非交互式shell，您应该打开一个终端来运行此命令。请查看[此处](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)以了解如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的计算机上运行下面的命令，它将提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有能够启动Web浏览器的计算机，例如笔记本电脑或工作站，才可以运行此命令。完成您的浏览器认证步骤后，您将需要将验证代码复制并粘贴回原始终端以继续登录流程。如果您在支持Web浏览器的计算机上运行此命令，则无需使用额外的计算机。

此命令将打印凭据文件为`凭据保存到文件：[CREDENTIALS_FILE]`。

In [ ]:
!gcloud auth application-default login

指定凭证文件的路径，该文件为json文件。

In [ ]:
CREDENTIALS_FILE = "[CREDENTIALS_FILE]"  # @param {type:"string"}

接下来，授权gcloud使用Google用户凭据访问云平台。用户凭据需要在项目中具有`setIamPolicy`权限。更多详细信息请查看[这里](https://cloud.google.com/resource-manager/docs/access-control-proj)。

与上一步类似，这也需要用户输入。上一步的说明在这里同样适用。

In [ ]:
!gcloud auth login

授予用户帐户角色。使用您在gcloud auth步骤中使用的电子邮件地址，例如`myemail@gmail.com`。由于您将使用此用户帐户从Google Cloud Storage下载模型工件，因此您授予用户帐户**存储对象查看器**角色。有关更多详细信息，请参阅[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
USER_ACCOUNT = "[USER_ACCOUNT]"  # @param {type:"string"}

!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=user:$USER_ACCOUNT --role=roles/storage.objectViewer

使用项目ID初始化Vertex AI SDK，此操作需要用户凭据。

In [ ]:
from google.cloud import aiplatform

aiplatform.init(project=PROJECT_ID, location=REGION)

选项2. 使用Google服务账号凭据

如果您已经完成了上述的选项1，请跳过这一步。首先，如果尚未启用IAM API，请先启用它。

In [ ]:
!gcloud services enable iam.googleapis.com

服务账户

您使用服务账户在本地运行容器时下载模型工件。如果您不想使用项目的计算引擎服务账户，可以将`SERVICE_ACCOUNT`设置为另一个服务账户ID。

In [ ]:
SERVICE_ACCOUNT = "[your-service-account]"  # @param {type:"string"}

In [ ]:
if (
    SERVICE_ACCOUNT == ""
    or SERVICE_ACCOUNT is None
    or SERVICE_ACCOUNT == "[your-service-account]"
):
    # Get your service account from gcloud
    if not IS_COLAB:
        shell_output = !gcloud auth list 2>/dev/null
        SERVICE_ACCOUNT = shell_output[2].replace("*", "").strip()

    else:  # IS_COLAB:
        shell_output = ! gcloud projects describe  $PROJECT_ID
        project_number = shell_output[-1].split(":")[1].strip().replace("'", "")
        SERVICE_ACCOUNT = f"{project_number}-compute@developer.gserviceaccount.com"

    print("Service Account:", SERVICE_ACCOUNT)

创建服务帐户。

In [ ]:
!gcloud iam service-accounts create $SERVICE_ACCOUNT

授权gcloud以Google用户凭据访问Cloud Platform。用户凭据需要在项目中具有`setIamPolicy`权限。更多详细信息请查看[这里](https://cloud.google.com/resource-manager/docs/access-control-proj)。

这一步涉及交互式提示，需要用户输入。如果您使用非交互式shell，应该打开终端来运行此命令。请查看[这里](https://jupyterlab.readthedocs.io/en/stable/user/terminal.html)了解如何在Vertex Workbench笔记本环境中打开终端。

请注意，如果您在没有窗口管理器/浏览器的计算机上运行以下命令，它会提示您运行命令`gcloud auth application-default login --remote-bootstrap="..."`。只有可以启动Web浏览器的计算机，例如笔记本电脑或工作站，才允许运行此命令。完成使用您的浏览器进行身份验证步骤后，您需要将验证代码复制并粘贴回原始终端以继续登录流程。如果您在支持Web浏览器的计算机上运行此命令，则无需使用单独的计算机。

In [ ]:
!gcloud auth login

将角色授予服务帐户。由于您将使用此服务帐户从Google Cloud Storage下载模型工件，您要授予该服务帐户**存储对象查看器**角色。有关更多详细信息，请参见[Cloud Storage的IAM角色](https://cloud.google.com/storage/docs/access-control/iam-roles)。

In [ ]:
!gcloud projects add-iam-policy-binding $PROJECT_ID \
    --member=serviceAccount:{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com \
    --role=roles/storage.objectViewer

指定服务账号密钥文件名。

In [ ]:
CREDENTIALS_FILE = "./credentials.json"

生成服务账号密钥文件。这将在本地运行自定义服务容器时使用。

In [ ]:
!gcloud iam service-accounts keys create $CREDENTIALS_FILE \
    --iam-account="{SERVICE_ACCOUNT}@{PROJECT_ID}.iam.gserviceaccount.com"

请在本地运行并发送请求至容器

通过自定义预测程序功能，很容易在本地测试容器。

在此示例中，容器执行预测请求和健康检查。

选项1. 传递本地路径

该选项要求`Predictor`中的`load`函数支持从GCS路径和本地路径加载模型文件。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{LOCAL_MODEL_ARTIFACTS_DIR}",
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

将预测的响应和其内容打印出来。

In [ ]:
print(predict_response, predict_response.content)

打印出健康检查的响应和内容。

In [ ]:
print(health_check_response, health_check_response.content)

也要打印出所有容器日志。您将看到容器启动、处理请求和关闭容器的日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

选项2. 通过GCS路径

如果您想通过GCS路径访问文件，您需要在上一步设置凭证，并在运行容器时传递凭据路径。服务账户应该具有“Storage Object Viewer”权限。

In [ ]:
with local_model.deploy_to_local_endpoint(
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
    credential_path=CREDENTIALS_FILE,
) as local_endpoint:
    predict_response = local_endpoint.predict(
        request_file=INPUT_FILE,
        headers={"Content-Type": "application/json"},
    )

    health_check_response = local_endpoint.run_health_check()

打印出预测的响应及其内容。

In [ ]:
print(predict_response, predict_response.content)

打印出健康检查响应及其内容。

In [ ]:
print(health_check_response, health_check_response.content)

还要打印出所有的容器日志。您将看到容器启动、处理请求和容器关闭的日志。

In [ ]:
local_endpoint.print_container_logs(show_all=True)

### 将容器推送到制品注册表

配置Docker以访问制品注册表。然后将您的容器镜像推送到您的制品注册表存储库。

打印出项目中所有已启用的服务。

In [ ]:
!gcloud services list

如果在您的项目中未启用“ artifactregistry.googleapis.com”，请在继续之前启用该API。

In [ ]:
!gcloud services enable artifactregistry.googleapis.com

In [ ]:
!gcloud artifacts repositories create {REPOSITORY} \
    --repository-format=docker \
    --location=$REGION

In [ ]:
!gcloud auth configure-docker {REGION}-docker.pkg.dev --quiet

使用SDK推送图像。

In [ ]:
local_model.push_image()

部署到Vertex AI

### 上传自定义容器模型

In [ ]:
from google.cloud import aiplatform

In [ ]:
aiplatform.init(project=PROJECT_ID, location=REGION)

使用LocalModel实例来上传模型。它会自动为您填充容器规格。

In [ ]:
model = aiplatform.Model.upload(
    local_model=local_model,
    display_name=MODEL_DISPLAY_NAME,
    artifact_uri=f"{BUCKET_URI}/{MODEL_ARTIFACT_DIR}",
)

在Vertex AI上部署模型
完成这一步后，模型就部署好了，可以进行在线预测。

In [ ]:
endpoint = model.deploy(machine_type="n1-standard-4")

发送预测

使用Python SDK

In [ ]:
endpoint.predict(instances=[[6.7, 3.1, 4.7, 1.5], [4.6, 3.1, 1.5, 0.2]])

### 使用REST

In [ ]:
ENDPOINT_ID = endpoint.name

In [ ]:
! curl \
-H "Authorization: Bearer $(gcloud auth print-access-token)" \
-H "Content-Type: application/json" \
-d @instances.json \
https://{REGION}-aiplatform.googleapis.com/v1/projects/{PROJECT_ID}/locations/{REGION}/endpoints/{ENDPOINT_ID}:predict

### 使用gcloud命令行界面

In [ ]:
!gcloud ai endpoints predict $ENDPOINT_ID \
  --region=$REGION \
  --json-request=instances.json

清理工作

要清理掉在这个项目中使用的所有谷歌云资源，您可以删除用于本教程的[谷歌云项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
# Undeploy model and delete endpoint
endpoint.delete(force=True)

# Delete the model resource
model.delete()

# Delete the container image from Artifact Registry
!gcloud artifacts docker images delete \
    --quiet \
    --delete-tags \
    {REGION}-docker.pkg.dev/{PROJECT_ID}/{REPOSITORY}/{IMAGE}

delete_bucket = False

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI